In [0]:
dbutils.secrets.listScopes() #checking available scope to fetch key

dbutils.secrets.list("adam-scope") #checking available key inside the scope - keyvault +secret

spark.conf.set("fs.azure.account.key.adamdestorage.dfs.core.windows.net", dbutils.secrets.get(scope="adam-scope", key="adam-key"))  #configure key

In [0]:
# Import necessary libraries
from pyspark.sql.functions import *

In [0]:
# Read data from the Bronze layer
df =(spark.read
     .option("header" , "true")
     .option("inferSchema" , "true")
     .option("delimiter" , ",") #delimeter for CSV , use tsv or \t for unstructured data
    .csv("abfss://bronze@adamdestorage.dfs.core.windows.net/public_transport_data"))

In [0]:
df.limit(5).toPandas()

,_c0,Gombak,Taman Melati,Wangsa Maju,Sri Rampai,Setiawangsa,Jelatek,Dato' Keramat,Damai,Ampang Park,KLCC,Kampung Baru,Dang Wangi,Masjid Jamek (KJL),Pasar Seni (KJL),KL Sentral (KJL),Bangsar,Abdullah Hukum,Kerinchi,Universiti,Taman Jaya,Asia Jaya,Taman Paramount,Taman Bahagia,Kelana Jaya,Lembah Subang,Ara Damansara,Glenmarie,Subang Jaya,SS 15,SS 18,USJ 7 (KJL),Taipan,Wawasan,USJ 21,Alam Megah,Subang Alam,Putra Heights (KJL),Sungai Buloh,Kampung Selamat,Kwasa Damansara,Kwasa Sentral,Kota Damansara,Surian,Mutiara Damansara,Bandar Utama,TTDI,Phileo Damansara,Pusat Bandar Damansara,Semantan,Muzium Negara,Pasar Seni (SBK),Merdeka,Bukit Bintang,Tun Razak Exchange (TRX),Cochrane,Maluri (SBK),Taman Pertama,Taman Midah,Taman Mutiara,Taman Connaught,Taman Suntex,Sri Raya,Bandar Tun Hussein Onn,Batu 11 Cheras,Bukit Dukung,Sungai Jernih,Stadium Kajang,Kajang
0,Gombak,0,3,7,9,15,17,18,19,21,22,24,26,28,29,32,34,37,38,40,44,46,49,52,54,57,59,62,63,67,69,72,73,76,78,81,84,88,66,64,62,59,56,53,50,48,45,42,38,36,32,30,31,34,36,39,40,44,46,48,51,53,56,58,60,63,67,68,71
1,Taman Melati,3,0,4,6,12,14,15,16,18,19,21,23,25,26,29,31,34,35,37,41,43,46,49,51,54,56,59,60,64,66,69,70,73,75,78,81,85,63,61,59,56,53,50,47,45,42,39,35,33,29,27,28,31,33,36,37,41,43,45,48,50,53,55,57,60,64,65,68
2,Wangsa Maju,7,4,0,2,8,10,11,12,14,15,17,19,21,22,25,27,30,31,33,37,39,42,45,47,50,52,55,56,60,62,65,66,69,71,74,77,81,59,57,55,52,49,46,43,41,38,35,31,29,25,23,24,27,29,32,33,37,39,41,44,46,49,51,53,56,60,61,64
3,Sri Rampai,9,6,2,0,6,8,9,10,12,13,15,17,19,20,23,25,28,29,31,35,37,40,43,45,48,50,53,54,58,60,63,64,67,69,72,75,79,57,55,53,50,47,44,41,39,36,33,29,27,23,21,22,25,27,30,31,35,37,39,42,44,47,49,51,54,58,59,62
4,Setiawangsa,15,12,8,6,0,2,3,4,6,7,9,11,13,14,17,19,22,23,25,29,31,34,37,39,42,44,47,48,52,54,57,58,61,63,66,69,73,51,49,47,44,41,38,35,33,30,27,23,21,17,15,16,19,21,24,25,29,31,33,36,38,41,43,45,48,52,53,56


In [0]:
##To filter MRT :
df.limit(31).toPandas().iloc[:, -31:] #"-" from the right

,Sungai Buloh,Kampung Selamat,Kwasa Damansara,Kwasa Sentral,Kota Damansara,Surian,Mutiara Damansara,Bandar Utama,TTDI,Phileo Damansara,Pusat Bandar Damansara,Semantan,Muzium Negara,Pasar Seni (SBK),Merdeka,Bukit Bintang,Tun Razak Exchange (TRX),Cochrane,Maluri (SBK),Taman Pertama,Taman Midah,Taman Mutiara,Taman Connaught,Taman Suntex,Sri Raya,Bandar Tun Hussein Onn,Batu 11 Cheras,Bukit Dukung,Sungai Jernih,Stadium Kajang,Kajang
0,66,64,62,59,56,53,50,48,45,42,38,36,32,30,31,34,36,39,40,44,46,48,51,53,56,58,60,63,67,68,71
1,63,61,59,56,53,50,47,45,42,39,35,33,29,27,28,31,33,36,37,41,43,45,48,50,53,55,57,60,64,65,68
2,59,57,55,52,49,46,43,41,38,35,31,29,25,23,24,27,29,32,33,37,39,41,44,46,49,51,53,56,60,61,64
3,57,55,53,50,47,44,41,39,36,33,29,27,23,21,22,25,27,30,31,35,37,39,42,44,47,49,51,54,58,59,62
4,51,49,47,44,41,38,35,33,30,27,23,21,17,15,16,19,21,24,25,29,31,33,36,38,41,43,45,48,52,53,56
5,49,47,45,42,39,36,33,31,28,25,21,19,15,13,14,17,19,22,23,27,29,31,34,36,39,41,43,46,50,51,54
6,48,46,44,41,38,35,32,30,27,24,20,18,14,12,13,16,18,21,22,26,28,30,33,35,38,40,42,45,49,50,53
7,47,45,43,40,37,34,31,29,26,23,19,17,13,11,12,15,17,20,21,25,27,29,32,34,37,39,41,44,48,49,52
8,45,43,41,38,35,32,29,27,24,21,17,15,11,9,10,13,15,18,19,23,25,27,30,32,35,37,39,42,46,47,50
9,44,42,40,37,34,31,28,26,23,20,16,14,10,8,9,12,14,17,18,22,24,26,29,31,34,36,38,41,45,46,49


In [0]:
##To filter LRT :
df.limit(37).toPandas().iloc[:,:38]

,_c0,Gombak,Taman Melati,Wangsa Maju,Sri Rampai,Setiawangsa,Jelatek,Dato' Keramat,Damai,Ampang Park,KLCC,Kampung Baru,Dang Wangi,Masjid Jamek (KJL),Pasar Seni (KJL),KL Sentral (KJL),Bangsar,Abdullah Hukum,Kerinchi,Universiti,Taman Jaya,Asia Jaya,Taman Paramount,Taman Bahagia,Kelana Jaya,Lembah Subang,Ara Damansara,Glenmarie,Subang Jaya,SS 15,SS 18,USJ 7 (KJL),Taipan,Wawasan,USJ 21,Alam Megah,Subang Alam,Putra Heights (KJL)
0,Gombak,0,3,7,9,15,17,18,19,21,22,24,26,28,29,32,34,37,38,40,44,46,49,52,54,57,59,62,63,67,69,72,73,76,78,81,84,88
1,Taman Melati,3,0,4,6,12,14,15,16,18,19,21,23,25,26,29,31,34,35,37,41,43,46,49,51,54,56,59,60,64,66,69,70,73,75,78,81,85
2,Wangsa Maju,7,4,0,2,8,10,11,12,14,15,17,19,21,22,25,27,30,31,33,37,39,42,45,47,50,52,55,56,60,62,65,66,69,71,74,77,81
3,Sri Rampai,9,6,2,0,6,8,9,10,12,13,15,17,19,20,23,25,28,29,31,35,37,40,43,45,48,50,53,54,58,60,63,64,67,69,72,75,79
4,Setiawangsa,15,12,8,6,0,2,3,4,6,7,9,11,13,14,17,19,22,23,25,29,31,34,37,39,42,44,47,48,52,54,57,58,61,63,66,69,73
5,Jelatek,17,14,10,8,2,0,1,2,4,5,7,9,11,12,15,17,20,21,23,27,29,32,35,37,40,42,45,46,50,52,55,56,59,61,64,67,71
6,Dato' Keramat,18,15,11,9,3,1,0,1,3,4,6,8,10,11,14,16,19,20,22,26,28,31,34,36,39,41,44,45,49,51,54,55,58,60,63,66,70
7,Damai,19,16,12,10,4,2,1,0,2,3,5,7,9,10,13,15,18,19,21,25,27,30,33,35,38,40,43,44,48,50,53,54,57,59,62,65,69
8,Ampang Park,21,18,14,12,6,4,3,2,0,1,3,5,7,8,11,13,16,17,19,23,25,28,31,33,36,38,41,42,46,48,51,52,55,57,60,63,67
9,KLCC,22,19,15,13,7,5,4,3,1,0,2,4,6,7,10,12,15,16,18,22,24,27,30,32,35,37,40,41,45,47,50,51,54,56,59,62,66


In [0]:
# Filter MRT data and write to Silver
mrt_data = spark.createDataFrame(df.limit(31).toPandas().iloc[:, -31:])
mrt_path = "abfss://silver@adamdestorage.dfs.core.windows.net/MRT"
mrt_data.write.mode("overwrite").parquet(mrt_path)

In [0]:
# Filter LRT data and write to Silver
lrt_data = spark.createDataFrame(df.limit(37).toPandas().iloc[:, :38])
lrt_path = "abfss://silver@adamdestorage.dfs.core.windows.net/LRT"
lrt_data.write.mode("overwrite").parquet(lrt_path)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import pandas as pd

In [0]:
##Read data from URL
import pandas as pd

URL_DATA = 'https://storage.data.gov.my/transportation/ridership_headline.parquet'

df = pd.read_parquet(URL_DATA)
if 'date' in df.columns: df['date'] = pd.to_datetime(df['date'])

display(df)

In [0]:
# Read data from URL
URL_DATA = 'https://storage.data.gov.my/transportation/ridership_headline.parquet'
df = pd.read_parquet(URL_DATA)

# Convert 'date' column to datetime if it exists
if 'date' in df.columns: 
    df['date'] = pd.to_datetime(df['date'])

# Display the Pandas DataFrame
display(df)

date,bus_rkl,bus_rkn,bus_rpn,rail_lrt_ampang,rail_mrt_kajang,rail_lrt_kj,rail_monorail,rail_mrt_pjy,rail_ets,rail_intercity,rail_komuter_utara,rail_tebrau,rail_komuter
2019-01-01T00:00:00Z,null,null,null,113357,114173,139634,35804,null,null,null,null,null,null
2019-01-02T00:00:00Z,null,null,null,182715,169316,274224,31859,null,null,null,null,null,null
2019-01-03T00:00:00Z,null,null,null,187904,175304,286469,31893,null,null,null,null,null,null
2019-01-04T00:00:00Z,null,null,null,198420,187891,304755,34121,null,null,null,null,null,null
2019-01-05T00:00:00Z,null,null,null,120773,112660,145036,29950,null,null,null,null,null,null
2019-01-06T00:00:00Z,null,null,null,101145,95913,120032,25342,null,null,null,null,null,null
2019-01-07T00:00:00Z,null,null,null,197569,184365,301290,31988,null,null,null,null,null,null
2019-01-08T00:00:00Z,null,null,null,196879,185920,304680,31792,null,null,null,null,null,null
2019-01-09T00:00:00Z,null,null,null,197314,188770,307069,32305,null,null,null,null,null,null
2019-01-10T00:00:00Z,null,null,null,198876,189818,310510,32057,null,null,null,null,null,null


In [0]:
# Convert Pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(df)

In [0]:
# Define output path
output_path = "abfss://bronze@adamdestorage.dfs.core.windows.net/ridership_data"

In [0]:
# Write Spark DataFrame to parquet format
spark_df.write.mode("append").parquet(output_path)